# A Working Example

This example shows :

1 - how to run the program for some video

2 - how to set some important parameters for the detection, fixing and post processing

3 - how to show the result

## The detection part
The detection part is seprarted by loading a pretrained YOLOv4 network. The training should be done separately before running this tracking program.

The library that load the structure of YOLO is part of the project (https://github.com/Tianxiaomo/pytorch-YOLOv4)

Using other detection trained model is also possible, it just need to be loaded within the class `YoloDetector` with making corresponding changes to its methods.


To load all the libraries shown below, the directory should be at the root of the project folder. If this is ran in the `docs` directory, then one can run

In [1]:
# NOTE :To add path of the main files

import sys
import os

Dir = os.getcwd().split('\\')[-1]

if Dir =='docs':
    sys.path.insert(1, '../offlinemot')
elif Dir == 'Offline_MOT':
    %cd 'docs'
    sys.path.insert(1, 'offlinemot')

In [2]:
from detection import YoloDetector

This class can take several input parameters, like the YOLO configuration file, the model file (in pytorch format) and lastly a flag to wheather to use GPU or CPU

In [3]:
detector  = YoloDetector('../model/yolov4-obj.cfg','../model/Yolov4_epoch300.pth',use_cuda=False,namesfile='../model/obj.names')

convalution havn't activate linear
convalution havn't activate linear
convalution havn't activate linear


Detection is only performed every *N* frame. Where *N* is set according to `config.py` file. The following is a detailed description of all the paramers and its effects.

**Note** A several testing with the parameters values maybe  needed for certain type of videos. Once a working set of values is found then all the similar videos can use the same set.


## Config file content


With this variable:  `detect_every_N`, the number of detections frequency is determined.
If high accuracy is needed then a value of 1 is optimal.
Bigger values are less accurate but faster to run.

### Detection and general parameters

Other paramters influencing the detection and the general settings are:

```python
    ### general paprmters
    draw = False
    detect_every_N = 2
    missing_thresh = 0.8
    use_cuda = False
    resize_scale = 0.4
    
    ### Detection paramters
    model_name = 'model/Yolov4_epoch300.pth'
    model_config = 'model/yolov4-obj.cfg'
    classes_file_name = 'model/obj.names'
    detect_thresh = 0.3 #Yolo detection
    # distance to the nearst match between detection and tracking
    # output in pixels
    dist_thresh = 25
    size_thresh = 25
    detect_scale = 4.0
    
```

- The darwing flag `darw`, whether to draw or not.

- The `missing_thresh`parameter is for determining when to delete an object if it keeps failing tracking and detetcion, it should be between 0 and 1, with 0 means never delete, and 1 delete on the first failure. A value of 0.9 means delete if 10% of result is failed, keep otherwise.

- The `use_cuda` flag is whether to use GPU for detection or CPU.

- The `resize_scale` is just for display (if `draw` flag is True), to determine how much the image should be rescaled.

- The `model_name` is for the trained model file name

- The `model_config` is for setting the parameters of the Yolo network, if a different structure is trained then a different file should be given here.

- The `class_file_name` is a text file contining the names of the pridected classes from the detection network. 


- The `detect_thresh` is used to put a threshold on YOLO probabilty output for each detected class. The lower this value is the more detections it will give but with less accuracy. 

- The `dist_thresh` is for matching each detection with already detected object, it shouldn't be too big because it represent the minmum distance to match. Otherwise, False matching will be given.

- The `size_thresh` is another distance but to the change in width and height between a detection and a nearby object. It is used because the same object from bird's eye view should have the same dimensions. 

- The `detect_scale` is for smaller objects detection. sometimes the drone is too high and the objects are small, so we need to *zoom in* to detect in a better way. one solution here is to detect in two levels: full scale image in the ordinary case and smaller proposed cropped areas. This parameter control how small these areas are. Higher values will make the areas bigger. 



### Fixing and smoothing

In `config.py` file the following are the parameters for the part of fixing the view. The first boolean is for whether to do fixing or not. It will slow the processing, so if the video is stationary without noise then no need for it.

```python
### fix view paramers
do_fix = False
fixing_dilation = 13
min_matches     = 15
```

The part for doing the smoothing is also included in the same file as follows. The first boolean is whether to do the smoothing or not, the other two are related to Savitzky-Golay filter from `scipy` library.

The last boolean flag `save_out_video` will save a new mp4 file in the `outputs`folder with the same name for the processed file if set to True.

```python
### Smoothing for post processing
do_smooth   = True
window_size = 7
polydegree  = 3
save_out_video = False
```

Other steps (along with smoothing) are included in the `postprocess.py` file, namely, the orientation calculation and interpolation of missing positions in the tracks. The orientation is calculated from the trajectory itself where the next point in the trajectory determines the current point heading.

### Background subtraction parameters

Many parmeters are avaliable for background subtraction, as follows,

```python

    ### background subtractor parameters
    bgs_history = 5
    bgs_threshold = 50
    bgs_shadows = True
    bgs_learning = 0.5
    bgs_erosion_size = 3
    bgs_min_area = 300
    bgs_broder_margin =  0.45    # bigger would give boxes near the detected boxes with yolo

```


The most important of them are:

- The `bgs_history` determines how many frames are used to calculate the background

- The `bgs_threshold` is realted to the sensativaty of the subtraction , lower values will give more sensativity to movements

- The `bgs_erosion_size` is the síze of the mask to perform erosion on the forground. higher values will give thiner objects

- The `bgs_min_area` determines the minmum area of pixels that will be considered as object, otherwise it will be deleted.

- THe `bgs_broder_margin` determines how mush overlapping is allowed between already detected objects and newly found forground. this number is considered a percentage of the detected object dimensions that objects are allowed to be in. For example a value of 0.5 will mean everywhere is allowed, because half of the dimension on all the objects means all the objects' area.



### Filering parameters

Additionally, in the `config` file, there's parameters that will determine when to delete the detected objects.

```python

    ### Filtering Objects:
    min_history = 100
    overlap_thresh = 0.7
```

- The `min_hisory` is the minmum number of frames that the objects will start to be tested for percentage of correct tracking (with `missing_thresh`).

- The `overlap_thresh` is for the minmum area percentage of overlapping between confirmed objects to delete one of them.



## How to run

To run the full program, you need to run the `main.py` script with the `-v` flag set to the name and directory of the video in the base directory of the project. 

Try to run the following command to see the result on the sample video:

`python main.py -v docs\sample.mp4`

## How to show result

After running the program on your video, a txt file will be saved in the **outputs** directory with the same name as the provided video.

*Note* if your videos are named with similar names (for example numbers), you should copy the last result to other folder before start tracking a new video, because any new file with similar name will overwrite the previous one without warning.


If you want to show the result with angles and smoothing for the sample video above, you can run the command: 

`python show_results.py -v docs\sample.mp4`

The directory after the -v flag is the same as the one from which the video were processed.

## Output structure

in the output text file, each line is structured as follows,

*Frame_id* ---- \[ top_left_x ----- top_left_y ----- width height\] ----- *class_id* ----- *track_id* ----- *angel*

Example of one line:

´39 [3748, 964, 169, 73] 2 5 138´
